In [9]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv(r"..\data\raw\airline_ontime_2023\Airline_Delay_Cause.csv")

In [ ]:
#dropping duplicates and rows with no flights
df = df.drop_duplicates()
df = df[df["arr_flights"] > 0]

In [ ]:
#fixing column data types
int_cols = ["year", "month", "arr_flights", "arr_del15", "arr_cancelled", "arr_diverted"]
float_cols = ["carrier_ct", "weather_ct", "nas_ct", "security_ct", "late_aircraft_ct",
              "arr_delay", "carrier_delay", "weather_delay", "nas_delay",
              "security_delay", "late_aircraft_delay"]

for col in int_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

for col in float_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0.0)

In [14]:
df["date"] = pd.to_datetime(df["year"].astype(str) + "-" + df["month"].astype(str) + "-01")

df["total_delay"] = (
    df["carrier_delay"] + df["weather_delay"] +
    df["nas_delay"] + df["security_delay"] + df["late_aircraft_delay"]
)

df["delay_ratio"] = df["arr_del15"] / df["arr_flights"]

df["cancel_ratio"] = df["arr_cancelled"] / df["arr_flights"]

In [15]:
#clipping negative delays
delay_cols = ["arr_delay","carrier_delay","weather_delay","nas_delay","security_delay","late_aircraft_delay"]
df[delay_cols] = df[delay_cols].clip(lower=0)

In [16]:
keep_cols = ["date","year","month","carrier","carrier_name","airport","airport_name",
             "arr_flights","arr_del15","arr_cancelled","arr_diverted",
             "total_delay","delay_ratio","cancel_ratio"]

df_clean = df[keep_cols]

In [18]:
df_clean.to_csv("../data/processed/airline_ontime_2023_clean.csv", index=False)

print(f"Preprocessing complete! Cleaned file saved with {df_clean.shape[0]} rows.")
print(df_clean.head())

Preprocessing complete! Cleaned file saved with 24468 rows.
        date  year  month carrier       carrier_name airport  \
0 2024-01-01  2024      1      9E  Endeavor Air Inc.     ABE   
1 2024-01-01  2024      1      9E  Endeavor Air Inc.     ABY   
2 2024-01-01  2024      1      9E  Endeavor Air Inc.     AEX   
3 2024-01-01  2024      1      9E  Endeavor Air Inc.     AGS   
4 2024-01-01  2024      1      9E  Endeavor Air Inc.     ALB   

                                        airport_name  arr_flights  arr_del15  \
0  Allentown/Bethlehem/Easton, PA: Lehigh Valley ...           80         16   
1             Albany, GA: Southwest Georgia Regional            3          0   
2           Alexandria, LA: Alexandria International           62          9   
3        Augusta, GA: Augusta Regional at Bush Field           74         15   
4                   Albany, NY: Albany International          101         23   

   arr_cancelled  arr_diverted  total_delay  delay_ratio  cancel_ratio  
0